## Создание базы данных в ClickHouse и загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import math
from clickhouse_driver.client import Client

In [ ]:
# Подключение к серверу
client = Client('', user='default', password='qwerty') 

## Создание базы данных и структуры таблиц

База данных имеет название `real_db` содержит таблицу:
* real_db.ayk - с информацией об ценах на аукционе


In [ ]:
# Создание базы данных
client.execute("CREATE DATABASE real_db")

[]

In [ ]:
# Создание таблицы, в которой будет храниться информация об аукционе
client.execute('''
    CREATE TABLE realty_db.ayk
    (
      com_id Int32,
      time DateTime,
      name String,
      name_pers String,
      vikup String,
      price_stav Int32,
      price_vikup Int32,
    ) 
    ENGINE = ReplacingMergeTree()
    PARTITION BY main_category
    ORDER BY com_id
    SETTINGS index_granularity=256
    ''')

[]

In [ ]:
# Делаем таблицу инфраструктуры распределенной
client.execute('''
    CREATE TABLE real_db.distr_auk as real_db.auk
    ENGINE = Distributed(bd_real_estate_cluster, real_db, infr, xxHash64(com_id))
    ''')

[]

## Загрузка данных в таблицы

In [ ]:
# Подготовка данных об аукционе, полученных в результате парсинга, к загрузке в ClickHouse
infr_for_loading = pd.read_csv('./data_gathering/full.csv')
infr_for_loading = infr_for_loading.astype({"com_id": np.int64})

infr_for_loading = infr_for_loading.fillna('')
rows = infr_for_loading.values.tolist()

for i in range(len(rows)):
    for j in range(len(rows[i])):
        if j == len(rows[i]) - 1:
            if rows[i][j] == '':
                rows[i][j] = []
            else:
                rows[i][j] = rows[i][j].split('[\'')[1].split('\']')[0].replace("'", "").split(',')

In [ ]:
# Загрузка данных об инфраструктуре в таблицу real_db.distr_auk
client.execute("INSERT INTO real_db.distr_auk VALUES", rows)

21069

In [ ]:
# Разрываем соединение с сервером
client.disconnect()